# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    break

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
#print(file_path_list)   
# for every filepath in the file path list 
for f in file_path_list:
   # print(f)
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute(""" 
    CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = {
    'class' : 'SimpleStrategy',
    'replication_factor' : 1
    }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the udacity
try:
    session.set_keyspace('udacity')
    
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4 
## The table artist_song_by_sessionid_item is modelled based on the requirement Query1 by having 
## ATTRIBUTES - sessionid , iteminsession, artist, songtitle, songlength
## PRIMARY KEY - sessionid, iteminsession
## PARTITION KEY - sessionid
## CLUSTERING KEY - iteminsession
## ORDER BY - iteminsession

drop_table_artist_song_by_sessionid_item = 'DROP TABLE IF EXISTS artist_song_by_sessionid_item'
create_table_artist_song_by_sessionid_item = '''CREATE TABLE IF NOT EXISTS artist_song_by_sessionid_item
                                                (
                                                  sessionid int
                                                , iteminsession int
                                                , artist text
                                                , songtitle text
                                                , songlength float
                                                , PRIMARY KEY(sessionid, iteminsession))
                                                '''
try:
    session.execute(drop_table_artist_song_by_sessionid_item)
    session.execute(create_table_artist_song_by_sessionid_item)
except Exception as e:
    print(e)
    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_by_sessionid_item (sessionid, iteminsession, artist, songtitle, songlength)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# Select statement to verify the table was populated with data correctly
query1 = "select artist, songtitle, songlength from artist_song_by_sessionid_item where sessionid = 338 and iteminsession = 4"

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
# use pretty table to display data in the tabluar form and include headings
t = PrettyTable(['Artist', 'Song Title', 'Song Length'])
for row in rows:
    t.add_row([row.artist,row.songtitle, str(row.songlength)])
print(t)

+-----------+---------------------------------+--------------------+
|   Artist  |            Song Title           |    Song Length     |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


#### Do a SELECT to verify that the data have been inserted into each table

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## The table artist_song_by_userid is modelled based on the requirement Query2 by having 
## ATTRIBUTES - serid , sessionid, iteminsession, artist, song text, firstname, lastname
## PRIMARY KEY - userid, sessionid
## PARTITION KEY - userid,sessionid
## CLUSTERING KEY - iteminsession
## ORDER BY - iteminsession

drop_table_artist_song_by_userid = 'DROP TABLE IF EXISTS artist_song_by_userid'
create_table_artist_song_by_userid = ''' CREATE TABLE IF NOT EXISTS artist_song_by_userid
                                          (
                                            userid int
                                          , sessionid int
                                          , iteminsession int
                                          , artist text
                                          , song text
                                          , firstname text
                                          , lastname text
                                          , PRIMARY KEY((userid, sessionid), iteminsession))
                                    '''


try:
    session.execute(drop_table_artist_song_by_userid)
    session.execute(create_table_artist_song_by_userid)
except Exception as e:
    print(e)                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_by_userid (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
query2 = 'SELECT artist, song, firstname, lastname from artist_song_by_userid where userid = 10 and sessionid =182'

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
# use pretty table to display data in the tabluar form and include headings
t = PrettyTable(['Artist', 'Song Title', 'First Name', 'Last Name'])
for row in rows:
    t.add_row([row.artist,row.song, row.firstname, row.lastname])
print(t)

+-------------------+------------------------------------------------------+------------+-----------+
|       Artist      |                      Song Title                      | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## The table username_by_song is modelled based on the requirement Query3 by having
## ATRIBUTES - song text, userid, firstname , lastname
## PRIMARY KEY - song, userid
## PARTITION KEY - song
## CLUSTERING KEY - userid
## ORDER BY - userid
drop_table_username_by_song = 'DROP TABLE IF EXISTS username_by_song'
create_table_username_by_song = ''' CREATE TABLE IF NOT EXISTS username_by_song
                                    (   song text
                                      , userid int
                                      , firstname text
                                      , lastname text
                                      , PRIMARY KEY (song, userid))

                                '''
#drop and recreate tables 
try:
    session.execute(drop_table_username_by_song)
    session.execute(create_table_username_by_song)
except Exception as e:
    print(e)

In [15]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO username_by_song (song, userid, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
query3 = "select firstname, lastname from username_by_song where song = 'All Hands Against His Own'"    

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
# use pretty table to display data in the tabluar form and include headings
t = PrettyTable(['First Name', 'Last Name'])
for row in rows:
    t.add_row([row.firstname,row.lastname])
print(t)

+------------+-----------+
| First Name | Last Name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions
try:
    session.execute(drop_table_artist_song_by_sessionid_item)
    session.execute(drop_table_artist_song_by_userid)
    session.execute(drop_table_username_by_song)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()